In [ ]:
import os
from pathlib import Path

import numpy as np
import pycolmap

In [ ]:
train_data = Path("../image-matching-challenge-2023/train")

output_data = Path("../outputs")

dataset = "heritage"

scene = "dioscuri"

model_name = "DISK+LG+sift+NN-rot"

In [ ]:
n_images = len(os.listdir(train_data / dataset / scene / "images"))

print(f"Number of images: {n_images}")

In [ ]:
gt_model = train_data / dataset / scene / "sfm" 
gt_model = pycolmap.Reconstruction(str(gt_model))

print(gt_model.summary())

In [ ]:
our_model = output_data / model_name / dataset / scene / "sparse"
our_model = pycolmap.Reconstruction(str(our_model))

print(our_model.summary())

# Align Models

In [ ]:
def backward_project(
    points_2d: np.ndarray, image: pycolmap.Image, camera: pycolmap.Camera, depth: np.ndarray
) -> np.ndarray:
    """Project array of 2D points into the 3D world space given a depth map.

    Args:
        points2d (np.ndarray): Array of 2D points with shape (n, 2).
        image (pycolmap.Image): Image from which the points are taken.
        camera (pycolmap.Camera): Camera associated with the image.
        depth (np.ndarray): Associated depth map with shape (n,).

    Returns:
        np.ndarray: Array of backward projected 3D points from the given 2D points.
    """
    p_world = np.array(camera.image_to_world(points_2d))
    p_world = np.stack([p_world[:, 0], p_world[:, 1], np.ones_like(p_world[:, 0])]) * depth
    p_world = np.array(image.transform_to_world(p_world.T))

    return p_world

def get_camera_poses(reconstruction) -> np.ndarray:
    """Extracts camera positions from reconstruction.

    Args:
        reconstruction: pycolmap.Reconstruction(/path)

    Returns:
        np.ndarray: of shape (N, 3)
    """
    cameras = reconstruction.cameras
    images = reconstruction.images

    N = len(images)
    camera_poses = np.zeros((N, 3))
    for i, k1 in enumerate(images.keys()):
        image_1 = images[k1]
        camera_1 = cameras[image_1.camera_id]
        camera_poses[i] = backward_project(
            points_2d=np.array([[0, 0]]),
            image=image_1,
            camera=camera_1,
            depth=0,
        )
    return camera_poses

In [ ]:
image_names = [img.name for img in gt_model.images.values()]
locations = get_camera_poses(gt_model)

_ = our_model.align_robust(image_names, locations, 6)

# Visualize the models

In [ ]:
from hloc.utils.viz_3d import init_figure, plot_points, plot_reconstruction, plot_camera_colmap

fig3d = init_figure()
args = dict(max_reproj_error=2.0, min_track_length=2, cs=1, cameras=True)
plot_reconstruction(fig3d, gt_model, color='rgba(255, 0, 0, 0.5)', name="gt", **args)
plot_reconstruction(fig3d, our_model, color='rgba(0, 255, 0, 0.5)', name="ours", **args)
fig3d.show()